# Tarea Individual 1

## Victor Alejandro Leiva Espinoza

Se debe implementar en Python las siguientes visualizaciones, en base al Dataset de Migraciones.

In [1]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
import plotly.express as px

In [ ]:
#Se carga el archivo excel
file = "MigrantesChile (2005-2016).xlsx"
df = pd.read_excel(file)

 1. Considere los 5 países europeos con más migrantes en total. Presente un lineplot con la
 evolución de cantidad de migrantes desde dichos países para cada año del periodo considerado.

In [3]:
# Filtrar solo Europa, sumar cantidad de migrantes por año y seleccionar los 5 valores Totales mayores
df_europe = df[df["Continent"] == "Europe"].copy()
year_columns = [year for year in range(2005, 2017)]
df_europe.loc[:, "Total"] = df_europe[year_columns].sum(axis=1)
df_europe_sorted = df_europe.sort_values(by="Total", ascending=False)
df_final = df_europe_sorted.head(5)

#Se graficará la evolución anual y cumulativa:

#ANUAL:
#Se transforma el df a un formato largo
df_annual = df_final.melt(id_vars=["Country"], value_vars=year_columns, 
                          var_name="Year", value_name="Migration")
df_annual["Year"] = df_annual["Year"].astype(int)

# Se define un Diccionario de traducción
country_mapping = {
    "Spain": "España",
    "France": "Francia",
    "Germany": "Alemania",
    "Italy": "Italia",
    "United Kingdom": "Reino Unido"
}

# Se crea la columna Translated_Country con los nombres traducidos
df_annual["Translated_Country"] = df_annual["Country"].map(country_mapping).fillna(df_annual["Country"])

# Graficar la migración anual, usando Translated_Country para la leyenda
fig_annual = px.line(df_annual, 
                     x="Year", 
                     y="Migration", 
                     color="Translated_Country",  # Usamos Translated_Country aquí
                     labels={'Migration': 'Migración', 'Year': 'Año', "Translated_Country": "País"}, 
                     title='Migración Anual de los 5 Países Europeos con más Migración a Chile en el Período 2005-2016')

# CUMULATIVO
#Mismo procedimiento anterior de formato largo y traducción en un df al que se le aplicó suma cumulativa
df_cumulative = df_final[year_columns].cumsum(axis=1)
df_cumulative["Country"] = df_final["Country"]
df_cumulative = df_cumulative.melt(id_vars=["Country"], value_vars=year_columns, 
                                    var_name="Year", value_name="Cumulative_Migration")
df_cumulative["Year"] = df_cumulative["Year"].astype(int)
df_cumulative["Translated_Country"] = df_cumulative["Country"].map(country_mapping).fillna(df_cumulative["Country"])

# Graficar la migración acumulada, usando Translated_Country para la leyenda
fig_cumulative = px.line(df_cumulative, 
                         x="Year", 
                         y="Cumulative_Migration", 
                         color="Translated_Country",  
                         labels={'Cumulative_Migration': 'Migración Acumulada', 'Year': 'Año', "Translated_Country": "País"}, 
                         title='Migración Acumulada de los 5 Países Europeos con más Migración a Chile en el Período 2005-2016')

# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig_annual),
    dcc.Graph(figure=fig_cumulative)
])

# Ejecutar la app en puerto 8050
if __name__ == '__main__':
    app.run_server(port=8050, debug=True)

 2. De los 5 países anteriores, elija los dos con más migrantes en total. Presente dos area plots, 
uno apilado y otro no apilado, para comparar la evolución de cantidad de migrantes entre ambos
 países para cada año del periodo considerado,

In [4]:
#Tomar los dos totales mayores del df europeo
df_top_2 = df_europe_sorted.head(2)

#GRÁFICO APILADO
#Se cambia a formato largo para tener una fila por año y país
df_annual = df_top_2.melt(id_vars=["Country"], value_vars=year_columns, 
                          var_name="Year", value_name="Migration")
df_annual["Year"] = df_annual["Year"].astype(int)

# Crear la columna Translated_Country con los nombres traducidos
df_annual["Translated_Country"] = df_annual["Country"].map(country_mapping).fillna(df_annual["Country"])

# Graficar la migración anual apilada, usando Translated_Country para la leyenda
fig_stack = px.area(df_annual, 
                    x="Year", 
                    y="Migration", 
                    color="Translated_Country", 
                    labels={'Migration': 'Migración', 'Year': 'Año', "Translated_Country": "País"}, 
                    title='Migración Anual (Apilada) de los 2 Países Europeos con más Migración a Chile en el Período 2005-2016',
                    line_shape='linear')

#GRÁFICO NO APILADO
# Graficar la migración anual no apilada usando px.area
fig_unstack = px.area(
    df_annual, 
    x="Year", 
    y="Migration", 
    color="Translated_Country",  # Usamos Translated_Country aquí
    labels={'Migration': 'Migración', 'Year': 'Año', "Translated_Country": "País"}, 
    title='Migración Anual (No Apilada) de los 2 Países Europeos con más Migración a Chile en el Período 2005-2016',
    line_shape='linear'
)

# Para cada traza, eliminamos la propiedad de apilamiento y forzamos el relleno a cero
for trace in fig_unstack.data:
    trace.pop('stackgroup', None)  # elimina el atributo 'stackgroup' si existe
    trace.fill = "tozeroy"          # rellena el área hasta el eje cero



# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig_stack),
    dcc.Graph(figure=fig_unstack)
])

# Ejecutar la app en el puerto 8051
if __name__ == '__main__':
    app.run_server(port=8050+1, debug=True)

3. Un histograma, con ajuste de bins, de migrantes para los países que tengan en total, a lo más,
 55000 migrantes.

In [5]:
# Calcular el total de migrantes para cada país
df["Total"] = df[year_columns].sum(axis=1)

# Filtrar países con total a lo más 55.000 migrantes
df_filtered = df[df["Total"] <= 55000]

# Crear el histograma con ajuste de 11 bins de ancho = 5000 
fig_hist = px.histogram(
    df_filtered, 
    x="Total", 
    nbins=11, 
    title="Distribución del Total de Migrantes a Chile por País con un máximo de 55000 Migrantes",
    labels={"Total": "Total de Migrantes"}
)

# Asegurar que los límites de los bins sean cada 5000 y cambiar nombre del eje y
fig_hist.update_layout(
    xaxis=dict(
        tickmode='linear',  # Modo lineal para el eje X
        dtick=5000,         # Intervalo de 5000
        tick0=0,            # Empieza desde 0
    ),
    yaxis=dict(
        title="Número de Países"  
    )
)

# Modificar el hover para que no muestre "count" sino "Número de países"
fig_hist.update_traces(
    hovertemplate="Total de Migrantes: %{x}<br>" +  # Total de Migrantes
                  "Número de países: %{y}<extra></extra>"  # Usamos %{y} para el conteo
)

# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig_hist)
])

#Ejecutar en el puerto 8052
if __name__ == '__main__':
    app.run_server(port=8050+2, debug=True)

 4. Un Bar chart para comparar los migrantes de Europa y Asia, en cada año considerado en el
 dataset.

In [6]:
# Crear un DataFrame solo con Europa y Asia
df_ea = df[df["Continent"].isin(["Europe", "Asia"])].copy()

# Convertir el DataFrame a formato largo (long format) para tener una fila por año y país
df_long = df_ea.melt(id_vars=["Continent"], value_vars=year_columns, 
                     var_name="Year", value_name="Migrants")
df_long["Year"] = df_long["Year"].astype(int)

# Agrupar por continente y año, sumando la cantidad de migrantes
df_grouped = df_long.groupby(["Continent", "Year"], as_index=False)["Migrants"].sum()

# Crear un diccionario para traducir "Europe" a "Europa"
continent_mapping = {
    "Europe": "Europa",
    "Asia": "Asia"
}

# Aplicar la traducción al DataFrame
df_grouped["Translated_Continent"] = df_grouped["Continent"].map(continent_mapping).fillna(df_grouped["Continent"])

# Definir un orden específico para los continentes por coherencia con el siguiente gráfico
continent_order = {"Europe": 0, "Asia": 1}

# Mapear el orden de los continentes
df_grouped["Continent_Order"] = df_grouped["Continent"].map(continent_order)
df_grouped = df_grouped.sort_values(by="Continent_Order", ascending=True)


# Crear un gráfico de barras agrupadas con la columna traducida
fig = px.bar(df_grouped, 
             x="Year", 
             y="Migrants", 
             color="Translated_Continent",  # Usamos la columna traducida
             barmode="group",
             labels={"Migrants": "Migrantes", "Year": "Año", "Translated_Continent": "Continente"},
             title="Migrantes de Europa y Asia hacia Chile por Año en el período 2005-2016")

# Asegurar que todos los años del eje X se vean claramente
fig.update_layout(
    xaxis=dict(
        tickmode='linear',  
        tick0=min(df_grouped["Year"]),  
        dtick=1  # Intervalo entre los ticks (un año)
    ),
    margin=dict(l=40, r=40)  # Márgenes sencillos para que la leyenda no se oculte en una pantalla de computador
)


# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

#Ejecutar en el puerto 8053
if __name__ == '__main__':
    app.run_server(port=8050+3, debug=True)


5. Un Pie chart de migrantes que compare la cantidad de migrantes en total de países europeos
 versus países asiáticos.

In [7]:
# Agrupar por continente y sumando la cantidad total de migrantes
df_grouped_continent = df_long.groupby("Continent", as_index=False)["Migrants"].sum()

# Crear el gráfico de Pie chart
fig_pie = px.pie(df_grouped_continent, 
                 names="Continent", 
                 values="Migrants", 
                 title="Migración Total de Europa y Asia hacia Chile en el Período 2005-2016",
                 labels={"Migrants": "Total de Migrantes", "Continent": "Continente"})

# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig_pie)
])

#Ejecutar en el puerto 8054
if __name__ == '__main__':
    app.run_server(port=8050+4, debug=True)



 6. Un box-and-whisker chart  con los tres países que más inmigrantes aportan en total

In [8]:
#Filtrar los tres primeros países según total de migrantes
top_3_countries = df.nlargest(3, 'Total')

# Convertir el DataFrame a formato largo (long format) para tener una fila por año y país
df_long = top_3_countries.melt(id_vars=["Country"], value_vars=[year for year in range(2005, 2017)], 
                                var_name="Year", value_name="Migrants")

# Crear el gráfico de caja y bigotes
fig = px.box(df_long, 
             x="Country", 
             y="Migrants", 
             title="Box-and-Whisker Chart de los 3 Países con Más Inmigrantes a Chile en el Período 2005-2016",
             labels={"Country": "País", "Migrants": "Migrantes"})

# Crear y configurar la app Dash
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

#Ejecutar en el puerto 8055
if __name__ == '__main__':
    app.run_server(port=8050+5, debug=True)
